In [30]:
import dspy
import copy
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, MIPRO
from dspy.evaluate import Evaluate
from dspy.teleprompt.ensemble import Ensemble
from datasets import load_dataset
import random
import re
import os

random.seed(0)

In [31]:
lm = dspy.HFClientVLLM(model="meta-llama/Meta-Llama-3-8B",
                       port=8080,
                       url="http://localhost")
dspy.settings.configure(lm=lm)
NUM_THREADS = 8

# Dataset

* Create a dataset using the dspy.Example class
* We will use the Ultra Feedback dataset
    * 1 instruction
    * 4 possible completions
    * all of them are rated by gpt4
* Our goal is to find a good prompt to get the best RM out of Llama3
* We will create a train, valid, and test dataset
* We will evaluate our model using exact match

In [32]:
dataset = load_dataset("openbmb/UltraFeedback")
all_data = []
for input_ in dataset["train"]:
    completions = sorted(
        input_["completions"], key=lambda x: x["overall_score"], reverse=True
    )
    # only take the top and bottom completions
    for i, chosen_completion in enumerate(completions[:1]):
        for rejected_completion in completions[-1:]:
            if (
                chosen_completion["overall_score"]
                == rejected_completion["overall_score"]
            ):
                continue
            if random.random() < 0.5:
                text1 = chosen_completion["response"]
                text2 = rejected_completion["response"]
                preference = "1"
            else:
                text1 = rejected_completion["response"]
                text2 = chosen_completion["response"]
                preference = "2"
                
            # llama 3 has a smart context window
            if len(text1) > 1524 or len(text2) > 1524 or len(input_["instruction"]) > 1524:
                continue
            
            all_data.append(
                dspy.Example(
                    **{
                        "instruction": input_["instruction"],
                        "text_1": text1,
                        "text_2": text2,
                        "preferred_text": preference,
                    }
                ).with_inputs("instruction", "text_1", "text_2")
            )
    if len(all_data) > 2000:
        break

random.shuffle(all_data)

valid = all_data[:100]
test = all_data[100:600]
train = all_data[600:]

/home/toolkit/.conda/envs/dspy/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
Average Metric: 46 / 72  (63.9):  72%|███████▏  | 72/100 [00:16<00:02, 13.00it/s]

In [33]:
def extract_pref(example, pred):
    pref_pred = re.search(r"\d+", pred["preferred_text"])
    if pref_pred:
        pref_pred = pref_pred.group()
    else:
        pref_pred = None
    return pref_pred


def em_metric(example, pred, trace=None, frac=1.0, verbose=False):
    pref_pred = extract_pref(example, pred)
    if verbose:
        print(f"Example: {example['preferred_text']}")
        print(f"Prediction: {pref_pred}")
    score = example["preferred_text"] == pref_pred
    if score is None:
        return False
    return score

    


eval_fn = Evaluate(devset=test, metric=em_metric, num_threads=NUM_THREADS, display_progress=True)

In [34]:
class Preference(dspy.Signature):
    instruction = dspy.InputField()
    text_1 = dspy.InputField()
    text_2 = dspy.InputField()
    preferred_text = dspy.OutputField(desc="Return the preferred text (1 or 2)", prefix="preferred_text:")

class PrefPredict(dspy.Module):
    def __init__(self):
        super().__init__()
        self.preference = dspy.Predict(
            Preference, max_tokens=3, temperature=0.1
        ) 

    def forward(self, instruction, text_1, text_2, *args, **kwargs):
        preferred = self.preference(
            instruction=instruction,
            text_1=text_1,
            text_2=text_2,
        )
        return preferred

In [35]:
pred = PrefPredict()(**train[0])
print(pred)
em_metric(train[0], pred, verbose=True)

Prediction(
    preferred_text='2'
)
Example: 2
Prediction: 2


True

In [36]:
lm.inspect_history(n=3)





Given the fields `instruction`, `text_1`, `text_2`, produce the fields `preferred_text`.

---

Follow the following format.

Instruction: ${instruction}

Text 1: ${text_1}

Text 2: ${text_2}

preferred_text: Return the preferred text (1 or 2)

---

Instruction: Given a passage, rewrite it in the present tense. Yesterday I went to the grocery store to buy some vegetables.

Text 1: I am going to the grocery store to buy some vegetables.

Text 2: Today I go to the grocery store to buy some vegetables.

preferred_text: 2







In [37]:
eval_fn(PrefPredict())

Average Metric: 225 / 500  (45.0): 100%|██████████| 500/500 [00:05<00:00, 90.73it/s]

Average Metric: 225 / 500  (45.0%)


45.0

# Bootstrapping Few Shot Example with Random Search

In [38]:
boot_fs = BootstrapFewShotWithRandomSearch(metric=em_metric, max_bootstrapped_demos=4, max_labeled_demos=4,
                                           num_threads=NUM_THREADS, max_rounds=1, num_candidate_programs=12)

preference_model = boot_fs.compile(PrefPredict(), trainset=train, valset=valid)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 12 candidate sets.


Average Metric: 42 / 100  (42.0): 100%|██████████| 100/100 [00:01<00:00, 97.18it/s]


Average Metric: 42 / 100  (42.0%)
Score: 42.0 for set: [0]
New best score: 42.0 for seed -3
Scores so far: [42.0]
Best score: 42.0


Average Metric: 60 / 100  (60.0): 100%|██████████| 100/100 [00:07<00:00, 12.63it/s]


Average Metric: 60 / 100  (60.0%)
Score: 60.0 for set: [4]
New best score: 60.0 for seed -2
Scores so far: [42.0, 60.0]
Best score: 60.0


...

Average Metric: 56 / 100  (56.0%)
Score: 56.0 for set: [4]
Scores so far: [42.0, 60.0, 54.0, 56.0, 61.0, 56.0, 63.0, 57.0, 55.0, 59.0, 62.0, 60.0, 62.0, 56.0]
Best score: 63.0
Average of max per entry across top 1 scores: 0.63
Average of max per entry across top 2 scores: 0.82
Average of max per entry across top 3 scores: 0.9
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0


  0%|          | 7/1401 [00:00<01:49, 12.70it/s]


Bootstrapped 4 full traces after 8 examples in round 0.


Average Metric: 59 / 100  (59.0): 100%|██████████| 100/100 [00:08<00:00, 11.56it/s]

Average Metric: 59 / 100  (59.0%)
Score: 59.0 for set: [4]
Scores so far: [42.0, 60.0, 54.0, 56.0, 61.0, 56.0, 63.0, 57.0, 55.0, 59.0, 62.0, 60.0, 62.0, 56.0, 59.0]
Best score: 63.0
Average of max per entry across top 1 scores: 0.63
Average of max per entry across top 2 scores: 0.82
Average of max per entry across top 3 scores: 0.9
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
15 candidate programs found.


In [39]:
os.makedirs('prompts', exist_ok=True)
for idx, prog in enumerate([x[-1] for x in preference_model.candidate_programs[:3]]):
    prog.save(f'prompts/preference_model_{idx}.json')

In [40]:
eval_fn(preference_model)

Average Metric: 317 / 500  (63.4): 100%|██████████| 500/500 [01:36<00:00,  5.18it/s]

Average Metric: 317 / 500  (63.4%)


63.4

# Using an ensemble

In [41]:
ensemble_optimizer = Ensemble(reduce_fn=dspy.majority)
programs = [x[-1] for x in preference_model.candidate_programs]
ensemble_preference_model = ensemble_optimizer.compile(programs[:3])
eval_fn(ensemble_preference_model)

Average Metric: 320 / 500  (64.0): 100%|██████████| 500/500 [00:58<00:00,  8.52it/s]

Average Metric: 320 / 500  (64.0%)


64.0

# Adding Chain of Thought

In [42]:
class CoTPrefPredict(dspy.Module):
    def __init__(self):
        super().__init__()
        self.preference = dspy.ChainOfThought(
            Preference
        ) 

    def forward(self, instruction, text_1, text_2, *args, **kwargs):
        preferred = self.preference(
            instruction=instruction,
            text_1=text_1,
            text_2=text_2,
        )
        return preferred

In [43]:
cot_preference_model = boot_fs.compile(CoTPrefPredict(), trainset=train, valset=valid)

Average Metric: 24 / 100  (24.0): 100%|██████████| 100/100 [00:23<00:00,  4.33it/s]


Average Metric: 24 / 100  (24.0%)
Score: 24.0 for set: [0]
New best score: 24.0 for seed -3
Scores so far: [24.0]
Best score: 24.0


Average Metric: 58 / 100  (58.0): 100%|██████████| 100/100 [00:39<00:00,  2.53it/s]


Average Metric: 58 / 100  (58.0%)
Score: 58.0 for set: [4]
New best score: 58.0 for seed -2
Scores so far: [24.0, 58.0]
Best score: 58.0


  1%|          | 8/1401 [00:12<37:07,  1.60s/it]


...

Average Metric: 58 / 100  (58.0%)
Score: 58.0 for set: [4]
Scores so far: [24.0, 58.0, 54.0, 55.0, 61.0, 68.0, 57.0, 60.0, 58.0, 61.0, 60.0, 64.0, 54.0, 58.0]
Best score: 68.0
Average of max per entry across top 1 scores: 0.68
Average of max per entry across top 2 scores: 0.86
Average of max per entry across top 3 scores: 0.96
Average of max per entry across top 5 scores: 0.97
Average of max per entry across top 8 scores: 0.99
Average of max per entry across top 9999 scores: 0.99


  0%|          | 7/1401 [00:11<37:47,  1.63s/it]


Bootstrapped 4 full traces after 8 examples in round 0.


Average Metric: 57 / 100  (57.0): 100%|██████████| 100/100 [00:40<00:00,  2.47it/s]

Average Metric: 57 / 100  (57.0%)
Score: 57.0 for set: [4]
Scores so far: [24.0, 58.0, 54.0, 55.0, 61.0, 68.0, 57.0, 60.0, 58.0, 61.0, 60.0, 64.0, 54.0, 58.0, 57.0]
Best score: 68.0
Average of max per entry across top 1 scores: 0.68
Average of max per entry across top 2 scores: 0.86
Average of max per entry across top 3 scores: 0.96
Average of max per entry across top 5 scores: 0.97
Average of max per entry across top 8 scores: 0.99
Average of max per entry across top 9999 scores: 0.99
15 candidate programs found.


In [44]:
eval_fn(cot_preference_model)

Average Metric: 326 / 500  (65.2): 100%|██████████| 500/500 [03:33<00:00,  2.34it/s]

Average Metric: 326 / 500  (65.2%)


65.2

In [45]:
for idx, prog in enumerate([x[-1] for x in cot_preference_model.candidate_programs[:3]]):
    prog.save(f'prompts/cot_preference_model_{idx}.json')

In [46]:
mipro = MIPRO(metric=em_metric, init_temperature=1.0, num_candidates=4)
kwargs = dict(num_threads=NUM_THREADS, display_progress=True, display_table=4)
mipro_preference_model = mipro.compile(student=PrefPredict(), trainset=train, num_trials=10, max_bootstrapped_demos=1, 
                                        max_labeled_demos=0, eval_kwargs=kwargs, requires_permission_to_run=False)



Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 1401 examples in dev set * 10 trials * # of LM calls in your program = (14010 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 4 * 1 lm calls in program = 14 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may consider:



  0%|          | 1/1401 [00:00<00:05, 274.91it/s]


Bootstrapped 1 full traces after 2 examples in round 0.


  0%|          | 1/1401 [00:00<00:05, 262.83it/s]


Bootstrapped 1 full traces after 2 examples in round 0.


  0%|          | 1/1401 [00:00<01:04, 21.81it/s]


Bootstrapped 1 full traces after 2 examples in round 0.


In [ ]:
eval_fn(mipro_preference_model)

# Additional ressources

*